In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
import os
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
from sklearn.model_selection import train_test_split
import random
import time
from torch.cuda.amp import GradScaler, autocast

# Set device and random seeds
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

print(f"Using device: {device}")

# Define broccoli variation classes
broccoli_variations = ['Florets', 'In-Context(Cooking)', 'Whole Crown']
num_classes = len(broccoli_variations)
broccoli_folder = r"C:\Users\ashsh\Downloads\Broccoli"

print(f"Broccoli Variation Classification")
print(f"Number of classes: {num_classes}")
print(f"Class names: {broccoli_variations}")

# Custom Dataset class for broccoli variations
class BroccoliVariationDataset(Dataset):
    def __init__(self, broccoli_folder, variations, transform=None, train=True, train_split=0.8):
        self.broccoli_folder = broccoli_folder
        self.variations = variations
        self.transform = transform
        self.class_to_idx = {variation: idx for idx, variation in enumerate(variations)}
        
        self.image_paths = []
        self.labels = []
        
        print(f"Creating {'train' if train else 'test'} dataset...")
        
        for variation in variations:
            variation_folder = os.path.join(broccoli_folder, variation)
            if os.path.exists(variation_folder):
                images_in_variation = []
                for img_name in os.listdir(variation_folder):
                    if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                        img_path = os.path.join(variation_folder, img_name)
                        images_in_variation.append(img_path)
                
                train_imgs, test_imgs = train_test_split(images_in_variation, 
                                                       train_size=train_split, 
                                                       random_state=42)
                
                if train:
                    selected_images = train_imgs
                else:
                    selected_images = test_imgs
                
                self.image_paths.extend(selected_images)
                self.labels.extend([self.class_to_idx[variation]] * len(selected_images))
                
                print(f"  {variation}: {len(selected_images)} images")
        
        print(f"Total {'train' if train else 'test'} images: {len(self.image_paths)}")
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        
        try:
            image = Image.open(img_path).convert('RGB')
        except Exception as e:
            image = Image.new('RGB', (224, 224), (0, 0, 0))
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

# Data transforms optimized for broccoli variations
train_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=20),  # Good rotation for broccoli
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.05),  # Preserve green tones
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    transforms.RandomPerspective(distortion_scale=0.1, p=0.3),  # Slight perspective changes
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create datasets
print("Creating broccoli variation datasets...")
train_dataset = BroccoliVariationDataset(broccoli_folder, broccoli_variations, 
                                        transform=train_transforms, train=True)
test_dataset = BroccoliVariationDataset(broccoli_folder, broccoli_variations, 
                                       transform=test_transforms, train=False)

# Create DataLoaders (no workers)
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"DataLoaders created: batch_size={batch_size}")

# Test batch loading
print("Testing batch loading...")
sample_batch = next(iter(train_loader))
print(f"✓ Batch test successful! Images: {sample_batch[0].shape}, Labels: {sample_batch[1].shape}")

# Model class
class BroccoliVariationClassifier(nn.Module):
    def __init__(self, num_classes=3, pretrained=True, freeze_backbone=True):
        super(BroccoliVariationClassifier, self).__init__()
        self.resnet = models.resnet18(pretrained=pretrained)
        num_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_features, num_classes)
        
        if freeze_backbone:
            for param in self.resnet.parameters():
                param.requires_grad = False
            for param in self.resnet.fc.parameters():
                param.requires_grad = True
            print("Backbone frozen - only training final layer")
        
    def forward(self, x):
        return self.resnet(x)

# Create model
print("Creating ResNet model for broccoli variations...")
model = BroccoliVariationClassifier(num_classes=num_classes, pretrained=True, freeze_backbone=True)
model = model.to(device)

# Print parameter info
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

# Training setup
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3, verbose=True)
scaler = GradScaler()

# Training function
def train_broccoli_model(model, train_loader, test_loader, criterion, optimizer, scheduler, num_epochs=10):
    print(f"Starting training for {num_epochs} epochs...")
    print("-" * 60)
    
    history = {'train_loss': [], 'train_acc': [], 'test_loss': [], 'test_acc': [], 'learning_rates': []}
    best_test_acc = 0.0
    best_model_state = None
    patience_counter = 0
    early_stop_patience = 5
    
    for epoch in range(num_epochs):
        epoch_start_time = time.time()
        
        # Training phase
        model.train()
        running_loss = 0.0
        running_corrects = 0
        
        print(f"Epoch {epoch+1}/{num_epochs}")
        
        for batch_idx, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            
            with autocast():
                outputs = model(inputs)
                loss = criterion(outputs, labels)
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            
            _, preds = torch.max(outputs, 1)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            
            if (batch_idx + 1) % 15 == 0:
                print(f"  Batch {batch_idx+1}/{len(train_loader)}, Loss: {loss.item():.4f}")
        
        train_loss = running_loss / len(train_loader.dataset)
        train_acc = running_corrects.double() / len(train_loader.dataset)
        
        # Testing phase
        model.eval()
        test_running_loss = 0.0
        test_running_corrects = 0
        
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                
                with autocast():
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                
                _, preds = torch.max(outputs, 1)
                test_running_loss += loss.item() * inputs.size(0)
                test_running_corrects += torch.sum(preds == labels.data)
        
        test_loss = test_running_loss / len(test_loader.dataset)
        test_acc = test_running_corrects.double() / len(test_loader.dataset)
        
        scheduler.step(test_acc)
        current_lr = optimizer.param_groups[0]['lr']
        
        # Save metrics
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc.item())
        history['test_loss'].append(test_loss)
        history['test_acc'].append(test_acc.item())
        history['learning_rates'].append(current_lr)
        
        # Save best model
        if test_acc > best_test_acc:
            best_test_acc = test_acc
            best_model_state = model.state_dict().copy()
            patience_counter = 0
        else:
            patience_counter += 1
        
        # Print results
        epoch_time = time.time() - epoch_start_time
        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
        print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}")
        print(f"LR: {current_lr:.6f}, Time: {epoch_time:.1f}s")
        print(f"Best Test Acc: {best_test_acc:.4f}")
        
        # Early stopping
        if patience_counter >= early_stop_patience:
            print(f"Early stopping - no improvement for {early_stop_patience} epochs")
            break
            
        print("-" * 60)
    
    # Load best model
    model.load_state_dict(best_model_state)
    print(f"Training completed! Best test accuracy: {best_test_acc:.4f}")
    return model, history

# Start training
print("Starting broccoli variation training...")
num_epochs = 10

trained_model, training_history = train_broccoli_model(
    model, train_loader, test_loader, criterion, optimizer, scheduler, num_epochs
)

# Save model
os.makedirs("saved_models", exist_ok=True)
model_save_path = "saved_models/broccoli_variation_classifier.pth"

torch.save({
    'model_state_dict': trained_model.state_dict(),
    'class_names': broccoli_variations,
    'num_classes': num_classes,
    'model_architecture': 'resnet18',
    'best_test_accuracy': max(training_history['test_acc']),
    'training_history': training_history
}, model_save_path)

print(f"\n=== TRAINING COMPLETED ===")
print(f"Model saved to: {model_save_path}")
print(f"Best test accuracy: {max(training_history['test_acc']):.4f}")
print(f"Classes: {broccoli_variations}")

# Function to load the model later
def load_broccoli_model(model_path, device):
    checkpoint = torch.load(model_path, map_location=device)
    model = BroccoliVariationClassifier(num_classes=checkpoint['num_classes'])
    model.load_state_dict(checkpoint['model_state_dict'])
    model = model.to(device)
    model.eval()
    return model, checkpoint['class_names'], checkpoint['training_history']

print(f"\nTo load this model later, use:")
print(f"model, class_names, history = load_broccoli_model('{model_save_path}', device)")


Using device: cuda
Broccoli Variation Classification
Number of classes: 3
Class names: ['Florets', 'In-Context(Cooking)', 'Whole Crown']
Creating broccoli variation datasets...
Creating train dataset...
  Florets: 800 images
  In-Context(Cooking): 800 images
  Whole Crown: 800 images
Total train images: 2400
Creating test dataset...
  Florets: 200 images
  In-Context(Cooking): 200 images
  Whole Crown: 200 images
Total test images: 600
DataLoaders created: batch_size=64
Testing batch loading...
✓ Batch test successful! Images: torch.Size([64, 3, 224, 224]), Labels: torch.Size([64])
Creating ResNet model for broccoli variations...
Backbone frozen - only training final layer


c:\Users\ashsh\Downloads\CV_Task\cv_env\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\ashsh\Downloads\CV_Task\cv_env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Total parameters: 11,178,051
Trainable parameters: 1,539
Starting broccoli variation training...
Starting training for 10 epochs...
------------------------------------------------------------
Epoch 1/10


c:\Users\ashsh\Downloads\CV_Task\cv_env\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
C:\Users\ashsh\AppData\Local\Temp\ipykernel_43652\1494828417.py:160: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
C:\Users\ashsh\AppData\Local\Temp\ipykernel_43652\1494828417.py:188: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


  Batch 15/38, Loss: 0.9786
  Batch 30/38, Loss: 0.8096


C:\Users\ashsh\AppData\Local\Temp\ipykernel_43652\1494828417.py:215: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Train Loss: 0.9265, Train Acc: 0.5675
Test Loss: 0.6468, Test Acc: 0.7833
LR: 0.001000, Time: 360.5s
Best Test Acc: 0.7833
------------------------------------------------------------
Epoch 2/10
  Batch 15/38, Loss: 0.6195
  Batch 30/38, Loss: 0.6614
Train Loss: 0.6362, Train Acc: 0.7767
Test Loss: 0.4878, Test Acc: 0.8267
LR: 0.001000, Time: 351.7s
Best Test Acc: 0.8267
------------------------------------------------------------
Epoch 3/10
  Batch 15/38, Loss: 0.5324
  Batch 30/38, Loss: 0.4931
Train Loss: 0.5243, Train Acc: 0.8271
Test Loss: 0.4042, Test Acc: 0.8700
LR: 0.001000, Time: 343.9s
Best Test Acc: 0.8700
------------------------------------------------------------
Epoch 4/10
  Batch 15/38, Loss: 0.5121
  Batch 30/38, Loss: 0.3942
Train Loss: 0.4623, Train Acc: 0.8467
Test Loss: 0.3792, Test Acc: 0.8683
LR: 0.001000, Time: 339.0s
Best Test Acc: 0.8700
------------------------------------------------------------
Epoch 5/10
  Batch 15/38, Loss: 0.4318
  Batch 30/38, Loss: 0.3